In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder
import pickle

In [32]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [33]:
#Drop the unnecessary columns

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [34]:
##Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [35]:
#OneHot Encodeing Geography columns

from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder()
geo_encoder = onehot_encoder.fit_transform(data[['Geography']])

In [36]:
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [37]:
geo_arr = geo_encoder.toarray()
geo_arr

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [38]:
onehot_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [39]:
geo_encoded_df = pd.DataFrame(geo_arr, columns=onehot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [40]:
#combine all columns to orginal data

data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [41]:
# save the encoders and scaler

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder.pkl', 'wb') as file:
    pickle.dump(onehot_encoder, file)


In [42]:
# divide the dataset into X and y

X = data.drop('Exited', axis=1)
y = data['Exited']

#split the data in training and testing
X_train, X_test, Y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#Scaling thw Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [43]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,0.0,1.0,0.0


In [44]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

###Ann implementation

In [45]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import datetime

In [46]:
X_train.shape[1]

12

In [48]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [49]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 64)                832       
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [50]:
##Another Way of using optimizers, loss, etc
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

In [51]:
model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])
'''
loss, opt are the variables that use when we have to customised or add aprameters during compile.~~~Mentioned above
'''

'\nloss, opt are the variables that use when we have to customised or add aprameters during compile.~~~Mentioned above\n'

In [52]:
#setup of tensorflow
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

log_dir='logs/fit' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [53]:
#setup Early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [54]:
history = model.fit(
    X_train, Y_train, validation_data=(X_test, y_test), epochs=50,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/50
250/250 [==============================] - 2s 5ms/step - loss: 0.4053 - accuracy: 0.8316 - val_loss: 0.3653 - val_accuracy: 0.8500
Epoch 2/50
250/250 [==============================] - 1s 4ms/step - loss: 0.3547 - accuracy: 0.8511 - val_loss: 0.3431 - val_accuracy: 0.8545
Epoch 3/50
250/250 [==============================] - 1s 5ms/step - loss: 0.3467 - accuracy: 0.8577 - val_loss: 0.3437 - val_accuracy: 0.8635
Epoch 4/50
250/250 [==============================] - 1s 4ms/step - loss: 0.3442 - accuracy: 0.8579 - val_loss: 0.3476 - val_accuracy: 0.8600
Epoch 5/50
250/250 [==============================] - 1s 4ms/step - loss: 0.3410 - accuracy: 0.8576 - val_loss: 0.3385 - val_accuracy: 0.8580
Epoch 6/50
250/250 [==============================] - 1s 4ms/step - loss: 0.3391 - accuracy: 0.8581 - val_loss: 0.3433 - val_accuracy: 0.8545
Epoch 7/50
250/250 [==============================] - 2s 7ms/step - loss: 0.3366 - accuracy: 0.8599 - val_loss: 0.3413 - val_accuracy: 0.8600
Epoch 

In [55]:
model.save('model.h5')

c:\Users\siddhu\OneDrive\Desktop\Desktop\genai\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [56]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [57]:
%tensorboard --logdir logs/fit

#Predicting the Churn

load the trained model, scale pickle file, for streamlit prediction

In [58]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import pickle

In [59]:
#load the trained model
model = load_model('model.h5')

#load the encoders and scalers
with open('onehot_encoder.pkl', 'rb') as file:
    onehot_encoder=pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender=pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler=pickle.load(file)

In [85]:
#example inpiut data

input_data={
    'CreditScore': 200,
    'Geography': 'Spain',
    'Gender': 'Female',
    'Age': 48,
    'Tenure': 1,
    'Balance': 4003,
    'NumOfProducts': 3,
    'HasCrCard': 2,
    'IsActiveMember': 1,
    'EstimatedSalary':20348
}

In [86]:
from sklearn.preprocessing import StandardScaler , LabelEncoder

geo_encoded = onehot_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder.get_feature_names_out(['Geography']))

c:\Users\siddhu\OneDrive\Desktop\Desktop\genai\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [87]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [88]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,200,Spain,Female,48,1,4003,3,2,1,20348


##has we can see "Gender" columns has categorical values. convert it into numerical values ~~ using -> "label_encoder_gender"

In [89]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,200,Spain,0,48,1,4003,3,2,1,20348


##concatinate one hot encoded for "Geography" columns into numerical values

In [90]:
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,200,0,48,1,4003,3,2,1,20348,0.0,0.0,1.0


what i have done above code is 1.drop the 'Geography' column and 2.concate the geo_encoded_df encoder for geography in numerical values

###Scaling the input data by using scaler encoder

In [91]:
input_scaled_df = scaler.transform(input_df)
input_scaled_df

array([[-4.68707758, -1.09499335,  0.86537308, -1.38944225, -1.15437857,
         2.53355998,  2.83875637,  0.97481699, -1.39238176, -0.99850112,
        -0.57946723,  1.73494238]])

###Predicting the churn

In [92]:
pred = model.predict(input_scaled_df)
pred

1/1 [==============================] - 0s 44ms/step


array([[0.20227422]], dtype=float32)

In [93]:
pred_probability = pred[0][0]
pred_probability

0.20227422

In [94]:
if pred_probability > 0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
